<a href="https://colab.research.google.com/github/jford49/stable-diffusion-tensorflow/blob/master/Keras_SD_img2img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/divamgupta/stable-diffusion-tensorflow
# The following has canvas functions
# https://colab.research.google.com/drive/1Bf-bNmAdtQhPcYNyC-guu0uTu9MYYfLu#scrollTo=EjVM0HqKxGaf

In [ ]:
from IPython.core.display import display_latex
from datetime import datetime
import imageio
import numpy as np
import os
from os import listdir, path
import string
from PIL import Image

!pip install tensorflow tensorflow_addons ftfy --upgrade --quiet
import tensorflow as tf

s = '''mask_enc_path = "/content/drive/My Drive/SDimg2img/InPaint.mask.wide.JPG"
mask = Image.open(mask_enc_path)
#print("img_out shape", mask.shape)
display(mask)

img_height=512
img_width=896
test_image = mask.resize((img_width, img_height))
print("test_image type", type(test_image))
#print("test_image shape", test_image.shape)

test_image_array = np.array(test_image, dtype=np.float32)[None,...,:3]
#test_image_array = np.asarray(test_image)

print("test_image_array type", type(test_image_array))
print("test_image_array shape", test_image_array.shape)
print("len(test_image_array.shape)", len(test_image_array.shape))
#display(test_image_array)

test_image_tensor = tf.cast((test_image_array / 255.0) * 2 - 1, tf.float32)
print("input_image_tensor shape", test_image_tensor.shape)

def my_elementwise_func(x):
  if x > 0:
    return 1
  else:
    return -1

def loop_map(inputs):
  in_var = inputs.numpy()
  print("in_var type", type(in_var))
  print("in_var shape", in_var.shape)
  print("len(in_var.shape)", len(in_var.shape))
  for i in range(0,in_var.shape[1]):
    for j in range(0,in_var.shape[2]):
      for k in range(0,in_var.shape[3]):
        in_var[0,i,j,k] = my_elementwise_func(in_var[0,i,j,k])

  return tf.convert_to_tensor(in_var)

result = loop_map(test_image_tensor)
print("result type", type(result))
print("result shape", result.shape)

stop
img_out =  np.asarray(test_image_array, dtype=np.uint8)[0,:,:,:]

display(img_out)
print("img_out shape", img_out.shape)
img_out = Image.fromarray(img_out)
display(img_out)

stop#'''

s = '''listT = [1,2,3,4,5,6]
print([9] + listT[:-1])
print(" ")

num_steps = 50
input_image_strength = .3
timesteps = np.arange(1, 1000, 1000 // num_steps)
print(timesteps)
timesteps = timesteps[: int(len(timesteps)*input_image_strength)]
print(timesteps)
input_img_noise_t = timesteps[ int(len(timesteps)*input_image_strength) ]
print(input_img_noise_t)
print(" ")
e = enumerate(timesteps)
listT = [*e][::-1]  #  argument-unpacking operator = *
print(listT)
stop'''
s = '''[9, 1, 2, 3, 4, 5]

[  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341
 361 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701
 721 741 761 781 801 821 841 861 881 901 921 941 961 981]
[  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281]
81

[(14, 281), (13, 261), (12, 241), (11, 221), (10, 201), (9, 181), (8, 161), (7, 141), (6, 121), (5, 101), (4, 81), (3, 61), (2, 41), (1, 21), (0, 1)] '''

In [ ]:
!pip install git+https://github.com/jford49/stable-diffusion-tensorflow --upgrade #--quiet

!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2


In [ ]:
from stable_diffusion_tf.stable_diffusion import StableDiffusion

# Must be multiples of 8
img_height=512
img_width=512#896#896#768#
generator = StableDiffusion(
    img_height=img_height,
    img_width=img_width,
    jit_compile=False,  # You can try True as well (different performance profile)
)

###Image Generator Control
The following UPPER CASE game time commands are available:
* **N** *n* — The **N**umber of calls model will execute. A larger value of *n* gives more precision. During execution the progress bar will show two numbers on the LH side, index and timestep. The index goes from 0 to *n* and the timestep goes from 1000 to 0.
* **B** *n* — The **B**atch size is the number of images simultaneously produced.
* **T** *x* — The **T**emperature [0, 1] is the noise amplitude used to corrupt any prompt image.  This is not used with text only prompts. A value of 0.8 will give a softer image while 0.85 jumps to the next level of detail. A value of 1 gives the most detail.
* **S** *x* — The input image **S**trength controls the image resolution. A higher value will have more detail. A value of 0.6 is a good painterly comprimise between resolution and image quality.
* **X** *x* — The **X** parameter causes the image to warp right if > 0 or left if < 0. Values with abs(*x*) > 1 can cause artifacts. Values with abs(*x*) < .1 work best.
* **Y** *x* — The **Y** parameter causes the image to warp up if > 0 or down if < 0. Values with abs(*x*) > 1 can cause artifacts. Values with abs(*x*) < .1 work best.
* **Z** *x* — The **Z** parameter causes the image to zoom in if > 1.  Values of **Z** < 1 can cause frame artifacts.  A single call with -1 < **Z** < 0 can create dark but interesting images.
* **R** *n* — The **R** parameter sets or clears image **R**ecursion.  The default value for **R** is 0 which prevents recursion.  **R** is always <= **B**.  For example, if **B**=3, then setting **R**=2 would use the 2nd image for recursion. If **R** is set to -1 any image previously set as the main prompt would be re-set and **R** would be set to 0.
* **SEED [FIX | FIX *n* | *n*]** — Reseeds with some integer *n*.  If absent, the current seed will be kept.  **R**ecursion with a fixed seed can create unpleasent contrast.
* **SEED RND** — A random seed will be used.
* **SINGLES [true | false]** — If this keyword is placed BEFORE any of the above commands all the AI images generated up to the next command prompt will be output to the *path_to_auto_save* if defined. Normally only images displayed with the prompt will be saved.
* **AUTO_MASK** — This is normally used in conjunction with a given mask image to overlay dark areas outside the main mask.  It also has an effect without a given main mask.
* **SAVE** — If **B**atch size > 0 any image can be optionally saved. Multiple commands can be put on a single line. This is independent of the default image auto save.
* **FREEZE** — If **B**atch size > 0

While the main text is updated by default without a parameter prompt prefix entry, the following UPPER CASE commands will update other text inputs:
* **A** *text* — Updates the **A**rtist(s) style phrase (ex: 'painted by vettriano').
* **I** *text* — Updates the **I**ncantation phrase (ex: lifelike, photorealistic, artstation, sharp focus).
* **--** *text* — Updates the negative prompt.

These additional text commands are for convienience only.

*Notes:*
1. Failing to use UPPER CASE for a command will cause that command to be interpreted as a text prompt (suprise!).

1. Multiple commands can be entered on a single prompt when seperated by a semi-colon.

1. Default value for **X**, **Y**, and **Z** is 1.  A value other than 1 will cause recursion on the prompt image independent of the **R** setting.

In [ ]:

img_dict = {}
cmd_cursor = 0
cmds = []
cmd_times = []

def seed_string(seed, seed_fixed, always_show_seed = False):
  ss = "SEED "
  if seed_fixed:
    return ss + "FIX " + str(seed)
  else:
    if always_show_seed:
      return ss + "RND " + str(seed)
    else:
      return ss + "RND "

def update_from_prompt(imeh):
  global t
  global prev_text_input
  global prev_N, prev_T, prev_Str, prev_R, prev_Sing, prev_Z, prev_X, prev_Y, prev_B, prev_Seed
  global prev_Seed_fixed, prev_auto_mask, prev_g_scale
  global num_steps, temperature, input_image_strength, recursive_image, singles, zoom, expX, expY, g_scale
  global batch_size, seed, seed_fixed, use_auto_mask
  global text_input0, text_input_I, text_input_A, text_input_neg
  global path_to_auto_run
  global cmd_cursor, cmds, save_cmds, cmd_times, img_dict

  if imeh != '':
    imeh = imeh.replace("=", " ")
    try:#if True:#
      # Only used for cmds from file
      if imeh.startswith('IGNORE'):
        print('IGNORE')
        return 0

      if imeh.startswith('''N'''):
        num_steps = int(imeh[2:])
        print('''PREV: N='''+ str(prev_N)+",", '''NEW: N='''+ str(num_steps))
        prev_N = num_steps
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''B'''):
        batch_size = int(imeh[2:])
        print('''PREV: B='''+ str(prev_B)+",", '''NEW: B='''+ str(batch_size))
        prev_B = batch_size
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''T'''):
        temperature = float(imeh[2:])
        print('''PREV: T='''+ str(prev_T)+",", '''NEW: T='''+ str(temperature))
        prev_T = temperature
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''SINGLES'''):
        singles = not singles
        print('''PREV: SINGLES='''+ str(prev_Sing)+",", '''NEW: SINGLES='''+ str(singles))
        prev_Sing = singles
        save_cmd(imeh)
        if singles:
          batch_size = 0
        else:
          batch_size = 1
        return 2
      elif imeh.startswith('''AUTO_MASK'''):
        use_auto_mask = not use_auto_mask
        print('''PREV: AUTO_MASK='''+ str(prev_auto_mask)+",", '''NEW: AUTO_MASK='''+ str(use_auto_mask))
        prev_auto_mask = use_auto_mask
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''SEED'''):
        parts = imeh.strip().split()
        if parts[1] == '''FIX''':
          if len(parts) == 2:
            seed_fixed = True
          else:
            seed = int(parts[2])
            seed_fixed = True
        elif parts[1] == '''RND''':
            seed_fixed = False
        else:
            seed = int(parts[1])
            seed_fixed = True

        print('''PREV: '''+seed_string(prev_Seed, prev_Seed_fixed)+",", '''NEW: '''+ seed_string(seed, seed_fixed))
        prev_Seed = seed
        prev_Seed_fixed = seed_fixed
        save_cmd(seed_string(seed, seed_fixed))
        return 2
      elif imeh.startswith('''S'''):
        iis = float(imeh[2:])
        if iis < 0 or iis > 1:
          print('''S='''+str(iis)+ ''' not in [0, 1]. No change.''')
          return 1
        else:
          input_image_strength = iis
          print('''PREV: S='''+ str(prev_Str)+",", '''NEW: S='''+ str(input_image_strength))
          prev_Str = input_image_strength
          save_cmd(imeh)
          return 2
      elif imeh.startswith('''R'''):
        recursive_image = int(imeh[2:])
        recursive_image = min(recursive_image, batch_size)
        print('''PREV: R='''+ str(prev_R)+",", '''NEW: R='''+ str(recursive_image))
        prev_R = recursive_image
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''Z'''):
        zoom = float(imeh[2:])
        print('''PREV: Z='''+ str(prev_Z)+",", '''NEW: Z='''+ str(zoom))
        prev_Z = zoom
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''X'''):
        expX = float(imeh[2:])
        print('''PREV: X='''+ str(prev_X)+",", '''NEW: X='''+ str(expX))
        prev_X = expX
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''Y'''):
        expY = float(imeh[2:])
        print('''PREV: Y='''+ str(prev_Y)+",", '''NEW: Y='''+ str(expY))
        prev_Y = expY
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''G'''):
        g_scale = float(imeh[2:])
        print('''PREV: G='''+ str(prev_g_scale)+",", '''NEW: G='''+ str(g_scale))
        prev_g_scale = g_scale
        save_cmd(imeh)
        return 2
      elif imeh.startswith('''I'''):
        text_input_I = imeh[2:]
        save_text(text_input_I, "Incantation")
        return 2
      elif imeh.startswith('''A'''):
        text_input_A = imeh[2:]
        save_text(text_input_A, "Artist")
        return 2
      else:
        if imeh.startswith('''RUN'''):
          file = imeh[3:].strip()
          if path.isdir(file):
            path_to_auto_run = file
            #load_auto_run_if()
            #save_init_cmds_if()
            #init_text_images()
            return 2
          else:
            print ('RUN Fail:', imeh)
            return 1
        elif imeh.startswith('''BREAK'''):
          path_to_auto_run == ''
          img_dict = {}
          cmd_cursor = 0
          cmds = []
          cmd_times = []
          return 2
        else:
          if (imeh[0].isupper() or imeh[0] in string.punctuation) and not imeh.startswith("--"):
            print ('Text Input Fail (Caps or punctuation):', imeh)
            return 1
          else:
            prev_text_input = text_input0
            save_cmd(imeh)
            if imeh.startswith("--"):
              text_input_neg = imeh.strip('-').strip(' ')
            else:
              if imeh.find("--") > 0:
                parts = imeh.split("--")
                if len(parts) == 0:
                  text_input0 = imeh
                else:
                  text_input0 = parts[0]
                  text_input_neg = parts[1].strip('-').strip(' ')
              else:
                  text_input0 = imeh
    except:#else:#
      print('''Exception:''', imeh)
      return 1

  return 0

In [ ]:
def zoom_at(img, x, y, zoom):
  w, h = img.size
  zoom2 = zoom * 2
  img = img.crop((x - w / zoom2, y - h / zoom2,
                    x + w / zoom2, y + h / zoom2))
  return img.resize((w, h), Image.LANCZOS)

def zoom_at_mid(img, zoom):
  w, h = img.size
  return zoom_at(img, w/2, h/2, zoom)

def zoom_horiz(img, zoom):
  w, h = img.size
  zoom2 = zoom * 2
  w_ = w/2 + w / zoom2
  if zoom > 0:
    img = img.crop((0, 0, w_, h))
  else:
    img = img.crop((w_, 0, w - w_, h))

  return img.resize((w, h), Image.LANCZOS)

def zoom_vert(img, zoom):
  w, h = img.size
  zoom2 = zoom * 2
  h_ = h/2 + h / zoom2
  if zoom > 0:
    img = img.crop((0, 0, w, h_))
  else:
    img = img.crop((0, h_, w, h - h_))

  return img.resize((w, h), Image.LANCZOS)

def save_image_text(folder, img, text):
  file = path.join(folder, text + '''.png''')
  imageio.imwrite(file, img)
  return file

def save_cmd(imeh):
  global itt
  global path_to_auto_save, save_cmds
  if not save_cmds:
    return

  imeh = imeh.replace('/','^')
  file = path.join(path_to_auto_save, str(itt)+'+CMD+'+imeh + '''.png''')
  imageio.imwrite(file, cmd_image)

def save_text(text, title):
  global itt
  global path_to_auto_save, save_cmds
  if not save_cmds:
    return

  file = path.join(path_to_auto_save, str(itt)+'+TXT+'+title + '''.txt''')
  with open(file, 'w') as f:
    f.write(text)

def remove_replace(pil_idx, itt, seeds, imgs):
  global path_to_auto_save, text_input0

  file = str(itt)+'.'+str(seeds[0])+'.'+text_input0
  if path.isfile(path.join(path_to_auto_save, file)):
    # Remove default save as
    os.remove(path.join(path_to_auto_save, file))

  file = str(itt)+'.'+str(seeds[pil_idx])+'.'+text_input0
  pil_img = Image.fromarray(imgs[pil_idx][0])
  save_image_text(path_to_auto_save, pil_img, file) # Replace default save as


In [ ]:
def img_to_latent(img):
  test_image = img.resize((img_width, img_height))
  test_image_array = np.array(test_image, dtype=np.float32)[None,...,:3]
  #print("input_image_array shape", test_image_array.shape)

  test_image_tensor = tf.cast((test_image_array / 255.0) * 2 - 1, tf.float32)
  #print("input_image_tensor shape", test_image_tensor.shape)
  latent = generator.encode(test_image_tensor)
  #print("latent type", type(latent))
  #print("latent shape", latent.shape)
  #print(latent.shape[1])
  return latent

img_enc_path = None
#img_enc_path = "/content/drive/My Drive/SDimg2img/img00009.orig.png"
img_enc_path = "/content/drive/My Drive/SDimg2img/img00009.NST.2.jpg"
#img_enc_path = "/content/drive/My Drive/SDimg2img/img00009.m2.jpg"
#img_enc_path = "/content/drive/My Drive/SDimg2img/00013.jpg"
#img_enc_path = "/content/drive/My Drive/SDimg2img/asian girl.jpg"
#img_enc_path = "/content/drive/My Drive/SDimg2img/1.KPV.png"

#img_enc_path = "/content/drive/My Drive/SDimg2img/coneTree.jpg"  # Very large original generates best detail.
#img_enc_path = "/content/drive/My Drive/SDimg2img/coneTree.sm.jpg"     # Smaller version of above generates less detail
                                                                        # and may give different result.
if img_enc_path != None:
  img = Image.open(img_enc_path)
  display(img)###################'''

mask_enc_path = None  # string, inpainting occurs in the BLACK area
s = '''mask_enc_path = "/content/drive/My Drive/SDimg2img/img00009.m2.jpg"
mask = Image.open(mask_enc_path)
display(mask)#####################'''

s = '''latent_img = img_to_latent(img)
latent_mask = img_to_latent(mask)
latent = latent_img * .5 + latent_mask * .5

def my_elementwise_func(x):
  if x > 0:
    return 1
  else:
    return 0

def loop_map(inputs):
  in_var = inputs.numpy()
  #print("in_var type", type(in_var))
  #print("in_var shape", in_var.shape)
  #print("len(in_var.shape)", len(in_var.shape))
  for i in range(0,in_var.shape[1]):
    for j in range(0,in_var.shape[2]):
      for k in range(0,in_var.shape[3]):
        in_var[0,i,j,k] = my_elementwise_func(in_var[0,i,j,k])

  return tf.convert_to_tensor(in_var)

result = latent#loop_map(latent)
img_out = generator.decode(result)
#print("img_out shape", img_out.shape)

img_out = Image.fromarray(img_out)
display(img_out)

stop#####################'''

In [ ]:
import time
import random

path_to_auto_run = ''
path_to_auto_save = "/content/drive/My Drive/SDimg2img/OUT"
if not path.isdir(path_to_auto_save):
  os.mkdir(path_to_auto_save)

cmd_image_path = '''/content/drive/My Drive/L3V CMD.JPG'''
cmd_image = imageio.imread(cmd_image_path)

itt = 0

################### Set initial parameters ############################
query_at_end = True
read_list = False
singles = False
feedback = False
use_auto_mask = False

seed = 2570 # Both global and operation seed
seed_list = [seed] #[6419,81054,4244,91687,72487]
seed = seed_list[0]
seed_fixed = True

img_path_list = None #["img00009.orig.png","img00010.orig.png","img00015.orig.png"] # None
num_steps_list=[13]
#for i in range(0,30):
#  num_steps_list.append(13)

temperature_list=[.85] #[.9,.95,.98]  # 0, .8, 1
input_image_strength_list=[0.6]
zoom_list = [1.00]
x_list = [1.00]
y_list = [1.00]
g_scale_list = [7.5]#[2, 4, 7.5, 10, 13]
recursive_image = 0
#######################################################################

batch_size = 1
if singles:
  batch_size = 0

input_image=img_enc_path
input_image_folder = "/content/drive/My Drive/SDimg2img/"

text_input_I = ""
text_input_A = ""
text_input_neg = ""

# Greg Rutkowski artstation
# Wojciech Siudmak
# Peter Mohrbacher
# Zdzisław Beksiński
# john currin
# Yuskavage
# John Lavery
# Mark Rothko
# Edward Burne-Jones
# Junji Ito
# by alphonse mucha

# The following I text should be used as lyrics in Lux Aeterna
#text_input_I = "diffuse lighting, fantasy, intricate, elegant, highly detailed, lifelike, photorealistic, digital painting, artstation, illustration, concept art, smooth, sharp focus"
#text_input_A = "art by john collier and albert aublet and krenz cushart and artem demura"
text_input_A = "painted by vettriano"

#Negative prompts:  ' --3D, artstation, cinematic, renderer, oversaturated, render, kitsch' and called it 'decrap'
#{inaccurate limbs, extra limbs, mutated fingers, large breasts}, {words, logo, signature, watermark}

#text_input0 = "firetruck rain figure 5 painted by charles demuth"
#text_input0 = "yellow firetruck city streets"
text_input0 = "crow dancing in an empty building"# "crow girl
#text_input0 = "watermelons painted by euan uglow"
#text_input0 = "kisterskyia painted by vettriano"#kisterskaya
#text_input0 = "japanese being of transcendent light"
#text_input0 = "curse of oak island knights templar -- saturation female"

#print('''Start OK?''')
#imeh = input().strip()

prev_N = num_steps_list[0]
prev_Str = input_image_strength_list[0]
prev_T = temperature_list[0]
prev_R = recursive_image
prev_Sing = singles
prev_Z = zoom_list[0]
prev_X = x_list[0]
prev_Y = y_list[0]
prev_g_scale = g_scale_list[0]
prev_B = batch_size
prev_Seed = seed
prev_Seed_fixed = seed_fixed
prev_auto_mask = use_auto_mask

now = datetime.now()
dt_string = now.strftime("%H:%M:%S")
print("time =", dt_string)
!nvidia-smi -L

print(" ")
print(text_input0)
if (text_input_I != ""):    # Invocation
  print("I=",text_input_I)
if (text_input_A != ""):    # Artist
  print("A=",text_input_A)
if (text_input_neg != ""):  # Negative prompts
  print("--",text_input_neg)

if input_image is not None:
  print("Img:",input_image)
if mask_enc_path is not None:
  print("Mask:",mask_enc_path)

print(" ")
print("Width:", img_width,", Height:", img_height)
print("SINGLES", str(singles))
print("FEEDBACK", str(feedback))
print("AUTO_MASK", str(use_auto_mask))

num_iterations = 0
num_iterations = max(len(num_steps_list), num_iterations)
num_iterations = max(len(temperature_list), num_iterations)
num_iterations = max(len(input_image_strength_list), num_iterations)
num_iterations = max(len(seed_list), num_iterations)
num_iterations = max(len(g_scale_list), num_iterations)

if read_list:
  if query_at_end:
    print("Reading", str(num_iterations), "from lists:")
  else:
    print("Reading from lists:")

  print("\tSEEDs", seed_list)
  print("\tN", num_steps_list)
  print("\tT", temperature_list)
  print("\tS", input_image_strength_list)
  print("\tX", x_list)
  print("\tY", y_list)
  print("\tZ", zoom_list)
  print("\tG", g_scale_list)

num_steps = num_steps_list[0]
temperature = temperature_list[0]
input_image_strength = input_image_strength_list[0]
zoom = zoom_list[0]
expX = x_list[0]
expY = y_list[0]
g_scale = g_scale_list[0]

save_cmds = False
while not read_list:
  print("Start:", "N="+ str(num_steps),", B="+ str(batch_size),", T="+ str(temperature),", S="+ str(input_image_strength),
      ", R="+ str(recursive_image),", X="+ str(expX),", Y="+ str(expY),", Z="+ str(zoom),", G="+ str(g_scale))
  print(seed_string(seed, seed_fixed))
  print("Enter edit cmd | Leave blank to start")
  imeh = input().strip()
  if (imeh == ''):
    break

  for sub_cmd in imeh.split(';'):
    rc = update_from_prompt(sub_cmd.strip())

  if batch_size > 1:
    expX = 1
    expY = 1
    zoom = 1

  print(' ')

save_cmds = True

if path_to_auto_save != '':
  if itt == 0:
    # Clear folder
    onlyfiles = [f for f in listdir(path_to_auto_save) if path.isfile(path.join(path_to_auto_save, f))]
    for file in onlyfiles:
      os.remove(path.join(path_to_auto_save, file))

  if input_image != None:
    save_image_text(path_to_auto_save, Image.open(input_image), str(itt)+'.input_image')

if (text_input_I != ""):
  save_text(text_input_I, "Incantation")
if (text_input_A != ""):
  save_text(text_input_A, "Artist")
if (text_input_neg != ""):
  save_text(text_input_neg, "Neg Prompts")

save_cmd("N "+str(prev_N))
save_cmd("T "+str(prev_T))
save_cmd("S "+str(prev_Str))
save_cmd("X "+str(prev_X))
save_cmd("Y "+str(prev_Y))
save_cmd("Z "+str(prev_Z))
save_cmd("R "+str(prev_R))
save_cmd("B "+str(prev_B))
save_cmd("G "+str(prev_g_scale))
save_cmd("SINGLES " + str(prev_Sing))
save_cmd(seed_string(seed, seed_fixed))
save_cmd("FEEDBACK" + str(feedback))
save_cmd("AUTO_MASK" + str(use_auto_mask))

while(True):
  if read_list:
    num_steps = num_steps_list[min(itt, len(num_steps_list)-1)]
    temperature = temperature_list[min(itt, len(temperature_list)-1)]
    input_image_strength = input_image_strength_list[min(itt, len(input_image_strength_list)-1)]
    zoom = zoom_list[min(itt, len(zoom_list)-1)]
    g_scale = g_scale_list[min(itt, len(g_scale_list)-1)]

  text_input = (text_input0 + "," + text_input_I + "," + text_input_A).strip(',')

  s='''if img_path_list != None:
    for idx in range(len(img_path_list)):
      input_image = img_path_list[idx]
      img_path = input_image_folder + input_image
      #if path_to_auto_save != '':
      #  save_image_text(path_to_auto_save, Image.open(img_path), str(itt)+'.'+input_image)
      input_image = img_path#################'''

  imgs = []
  seeds = []
  for i in range(batch_size):
    if not seed_fixed:
      seed = random.randrange(1,99999)  # This is not a tf random
    if seed_list != None and read_list:
      seed = seed_list[min(itt, len(seed_list)-1)]

    gen_img = generator.generate_from_seed(
        text_input,
        negative_prompt = text_input_neg,
        num_steps = num_steps,
        unconditional_guidance_scale = g_scale, # https://huggingface.co/spaces/stabilityai/stable-diffusion/discussions/9
        temperature = temperature,
        batch_size = 1,
        seed = seed,
        input_image = input_image,  # string
        input_mask = mask_enc_path, # string, inpainting occurs in the BLACK area
        input_image_strength = input_image_strength,
        feedback = feedback,
        use_auto_mask = use_auto_mask
    )

    imgs.append(gen_img[0])
    seeds.append(seed)

  s='''pil_img = Image.fromarray(imgs[0][0])
      display(pil_img)
      save_image_text(path_to_auto_save, pil_img, str(itt)+'.'+str(seed)+'.'+str(idx)+'.'+text_input0)##############'''

  #print("len(imgs)", len(imgs))
  itt+=1

  first = 0
  #if mask_enc_path is None:
  #  first = max(0, len(imgs)-4)

  if singles:
    print("\nSINGLES " + str(first + 1))
    pil_img = Image.fromarray(imgs[first][0])
    display(pil_img)
    print("to ", len(imgs) - 1)
    pil_img = Image.fromarray(imgs[len(imgs) - 1][0])
    display(pil_img)
  else:
    for j in range(len(imgs)):
      if (batch_size > 1):
        print(str(j + 1), "-", str(seeds[j]), "vvvv")
      pil_img = Image.fromarray(imgs[j][0])
      display(pil_img)

  if (itt >= num_iterations and read_list):
    print("STOPPED reading from lists")
    read_list = False

  # Want the auto save file to reflect the latest images before input()
  if path_to_auto_save != '':
    if singles:
      for i in range(first, len(imgs)):
        pil_img = Image.fromarray(imgs[i][0])
        if (imgs[i][1] == ""):
          save_image_text(path_to_auto_save, pil_img, str(itt)+'.'+str(i)+'.'+str(seeds[0])+'.'+text_input0)
        else:
          save_image_text(path_to_auto_save, pil_img, str(itt)+'.'+str(i)+'.'+str(seeds[0])+'.'+imgs[i][1])
    else:
      # Default save as. Will be overwritten if recursive_image > 1
      save_image_text(path_to_auto_save, pil_img, str(itt)+'.'+str(seeds[0])+'.'+text_input0)####################'''

  if batch_size > 1:
    print(str(itt)+":", "N="+ str(num_steps),", B="+ str(batch_size),", T="+ str(temperature),", S="+ str(input_image_strength),
        ", R="+ str(recursive_image),", X="+ str(expX),", Y="+ str(expY),", Z="+ str(zoom),", G="+ str(g_scale), "   ^^^^^^^^^^^^^^^^^^^")
  else:
    print(str(itt)+":", "N="+ str(num_steps),", B="+ str(batch_size),", T="+ str(temperature),", S="+ str(input_image_strength),
        ", R="+ str(recursive_image),", X="+ str(expX),", Y="+ str(expY),", Z="+ str(zoom),", G="+ str(g_scale), ", "+seed_string(seeds[0], seed_fixed, True)+"   ^^^^^^^^^^^^^^^^^^^")

  print(text_input0)
  if (text_input_I != ""):
    print("I=",text_input_I)
  if (text_input_A != ""):
    print("A=",text_input_A)
  if (text_input_neg != ""):
    print("--",text_input_neg)

  #if input_image != None:
  #  print(input_image)

  imeh = ""
  if ((query_at_end and itt >= num_iterations) or not query_at_end):
    bNeedRepeat = True
    print(" ")
    while bNeedRepeat:
      print('''New text_input | params | blank for NEXT.''')

      sTime = batch_size * 2
      if (img_width > 512):
        sTime = 6
      time.sleep(sTime)

      imeh = input().strip()

      bNeedRepeat = False
      if batch_size > 1:
        sub_cmd0 = imeh.split(';')[0].strip()
        if sub_cmd0.startswith("FREEZE"):
          bNeedRepeat = True
          parts = sub_cmd0.strip().split()
          if len(parts) == 2:
            i = int(parts[1]) - 1
            if i < batch_size:
              if recursive_image > 1:
                remove_replace(i, itt, seeds, imgs)
              recursive_image = 0
              batch_size = 1
              seed_fixed = True
              seed = seeds[i]
              print(str(i + 1),"FROZEN:", "N="+ str(num_steps),", B=1, T="+ str(temperature),", S="+ str(input_image_strength),
                ", R=0, X="+ str(expX),", Y="+ str(expY),", Z="+ str(zoom),", G="+ str(g_scale), ", "+seed_string(seed, seed_fixed, True))
            else:
              print("Invalid FREEZE", i + 1, "cmd")
          else:
            print("Invalid FREEZE cmd")

      if not bNeedRepeat:
        bNeedRepeat = False
        ok = True
        #print(imeh.split(';'))
        for sub_cmd in imeh.split(';'):
          if sub_cmd.startswith("SAVE") and path_to_auto_save != '' and batch_size > 1:
            parts = sub_cmd.strip().split()
            if len(parts) == 2:
              i = int(parts[1])
              if i < batch_size:
                file = str(itt)+'.'+str(i)+'.'+str(seeds[i-1])+'.'+"SAVED"+'.'+imgs[i-1][1]
                save_image_text(path_to_auto_save, pil_img, file)
                print("SAVED:", file)
              else:
                print("Invalid SAVE cmd")
            else:
              print("Invalid SAVE cmd")
          else:
            rc = update_from_prompt(sub_cmd.strip())
            if rc >= 1:
              if read_list:
                print("STOPPED reading from lists")
              read_list = False

        if batch_size > 1 and seed_fixed:
          print("Error:","B="+str(batch_size),", "+seed_string(seed, seed_fixed))
          bNeedRepeat = True

    if batch_size > 1:
      expX = 1
      expY = 1
      zoom = 1

  if not singles:
    if (recursive_image > 1 and batch_size > 1):
      pil_idx = min(recursive_image - 1, batch_size - 1)
      remove_replace(pil_idx, itt, seeds, imgs)

  imageio.imwrite("lastImg.jpg", pil_img)

  if recursive_image > 0:
    input_image = "Img"+str(itt)+".jpg"
    imageio.imwrite(input_image, pil_img)
  elif recursive_image < 0:
    input_image = img_enc_path
    recursive_image = 0

  if zoom != 1:
    pil_img = zoom_at_mid(pil_img, zoom)
    input_image = "ZImg"+str(itt)+".jpg"
    imageio.imwrite(input_image, pil_img)

  # Negative values warp right
  if expX != 1 and expX != -1:
    pil_img = zoom_horiz(pil_img, expX)
    input_image = "XImg"+str(itt)+".jpg"
    imageio.imwrite(input_image, pil_img)

  if expY != 1 and expY != -1:
    pil_img = zoom_vert(pil_img, expY)
    input_image = "YImg"+str(itt)+".jpg"
    imageio.imwrite(input_image, pil_img)


